# Portfolio Assignment week 02

This week's focus is on manifold learning and text clustering. As part of the portfolio assignment, you are required to make a contribution to either the manifold learning case or the text clustering case. There are several options for your contribution, so you can choose the one that aligns with your learning style or interests the most


### Manifold learning

Study the Tutorial tutorial_manifold_tSNE and the tutorial_manifold_spectral_clustering and the Study_Case_pipeline. Next improve the code by comparing the performance of k-means and spectral clustering. Also compare PCA and t-SNE in the visualization of the result. You can use the pipeline function of scikit-learn and hyperparameter tuning with GridSearchCV. Here's a possible approach:

- Load the dataset to be used for the clustering analysis.
- Preprocess the dataset as needed (e.g., scale the features, normalize the data, etc.).
- Define a pipeline with preprocessing and clustering
- use PCA and t-SNE for dimension reduction and visualize the dimensions, use the clusters to color the datapoints
- use GridSearchCV to optimize the hyper parameters
- Evaluate the performance of the models using a suitable metric
- choose the best cluster method and the best visualization method combination

Explain choises and evaluate outcome. You can do this assignment in pairs but if you do so mention each others name. Do not forget to reference. If you cannot figure out how to use GridSearchCV and or a pipeline, use your own solution


### Text clustering

Read, execute and analyse the code in the notebook tutorial_clustering_words. Then *choose one* of the assignments a), b) or c). 

a) read the article Clinical Documents Clustering Based on Medication/Symptom Names Using Multi-View Nonnegative Matrix Factorization. you can find the article <a href = 'https://pubmed.ncbi.nlm.nih.gov/26011887/'> here</a>. Explain the similarities of this notebook and the article. Explain in your own words what need to be added to this notebook to reproduce the article. There is no need to code the solution, you can mention in your own words the steps. 

b) Improve the outcome improving the data preprocessing and the hyper parameter configurations. Explain your choices. Your solution should be a coded solution with comments. Are there any other weighting solutions next to TF-IDF?

c) Provide a text clustering solution with your own data of interest, you can follow a similar approach to the one in the tutorial_clustering_words notebook. 

Mind you that you are not allowed to copy code solutions without referencing.

Manifold learning and text clustering

## Part A: Load and clean the text

### the dataset used here is from: https://www.kaggle.com/code/ashokrajuyadav/biomedical-text-calssification/input

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.feature_extraction import text
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
import re
import string
import glob
from sklearn.feature_extraction.text import CountVectorizer
import string
data_biomedical = pd.read_csv("datasets_DS3\\alldata_1_for_kaggle.csv", encoding="latin-1")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
data_biomedical

,Unnamed: 0,0,a
0,0,Thyroid_Cancer,Thyroid surgery in children in a single insti...
1,1,Thyroid_Cancer,""" The adopted strategy was the same as that us..."
2,2,Thyroid_Cancer,coronary arterybypass grafting thrombosis ï¬b...
3,3,Thyroid_Cancer,Solitary plasmacytoma SP of the skull is an u...
4,4,Thyroid_Cancer,This study aimed to investigate serum matrix ...
...,...,...,...
7565,7565,Colon_Cancer,we report the case of a 24yearold man who pres...
7566,7566,Colon_Cancer,among synchronous colorectal cancers scrcs rep...
7567,7567,Colon_Cancer,the heterogeneity of cancer cells is generally...
7568,7568,Colon_Cancer,"""adipogenesis is the process through which mes..."


In [3]:
data_biomedical['0'].value_counts()

Thyroid_Cancer    2810
Colon_Cancer      2580
Lung_Cancer       2180
Name: 0, dtype: int64

In [4]:
data_biomedical.rename(columns={"Unnamed: 0": "index", "0": "disease", "a": "text"}, inplace=True)

In [5]:
data_biomedical.set_index("index", inplace=True)

In [6]:
data_biomedical

,disease,text
index,,
0,Thyroid_Cancer,Thyroid surgery in children in a single insti...
1,Thyroid_Cancer,""" The adopted strategy was the same as that us..."
2,Thyroid_Cancer,coronary arterybypass grafting thrombosis ï¬b...
3,Thyroid_Cancer,Solitary plasmacytoma SP of the skull is an u...
4,Thyroid_Cancer,This study aimed to investigate serum matrix ...
...,...,...
7565,Colon_Cancer,we report the case of a 24yearold man who pres...
7566,Colon_Cancer,among synchronous colorectal cancers scrcs rep...
7567,Colon_Cancer,the heterogeneity of cancer cells is generally...


In [7]:
# stop_words = set(stopwords.words('english'))

# def text_cleaning(text):
#     # Remove punctuation and convert to lowercase
#     text = ''.join([char.lower() if char not in string.punctuation else ' ' for char in text])
    
#     # Split the text into tokens
#     tokens = text.split()
    
#     # Remove stopwords
#     filtered_tokens = [word for word in tokens if word not in stop_words]
    
#     return filtered_tokens

In [8]:
stopwords = set(stopwords.words('english'))

## Function used for cleaning

In [9]:
def text_cleaning(text):
    text = ''.join([word.lower() for word in text if word not in string.punctuation])
    tokens = text.split()
    text = [word for word in tokens if word not in stopwords]
    return text

In [10]:
data_biomedical.head()

,disease,text
index,,
0,Thyroid_Cancer,Thyroid surgery in children in a single insti...
1,Thyroid_Cancer,""" The adopted strategy was the same as that us..."
2,Thyroid_Cancer,coronary arterybypass grafting thrombosis ï¬b...
3,Thyroid_Cancer,Solitary plasmacytoma SP of the skull is an u...
4,Thyroid_Cancer,This study aimed to investigate serum matrix ...


### The code above is taken from: https://www.kaggle.com/code/ashokrajuyadav/biomedical-text-calssification/notebook

#### The next step is applied to remove read errors, punctuations, square brackets and symbols, words containing numbers and making text in lowercase for further processing in section B

In [11]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets, 
    remove punctuation, remove read errors,
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
     # Remove non-alphanumeric characters and extra space
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    text = re.sub('\w*\d\w*', ' ', text)
    text = re.sub('�', ' ', text)
    return text

cleaned = lambda x: clean_text(x)

#### the next part is to use wordnetlemmatizer extract and reduce words to its base root form and then determine whether its a noun or a verb

In [12]:
# Noun extract and lemmatize function
def nouns(text):
    '''Given a string of text, tokenize the text 
    and pull out only the nouns.'''
    # create mask to isolate words that are nouns
    is_noun = lambda pos: pos[:2] == 'NN'
    # store function to split string of words 
    # into a list of words (tokens)
    tokenized = word_tokenize(text)
    # store function to lemmatize each word
    wordnet_lemmatizer = WordNetLemmatizer()
    # use list comprehension to lemmatize all words 
    # and create a list of all nouns
    all_nouns = [wordnet_lemmatizer.lemmatize(word) \
    for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    
    #return string of joined list of nouns
    return ' '.join(all_nouns)

In [13]:
data_biomedical["text"] = data_biomedical["text"].apply(cleaned)
data_nouns = pd.DataFrame(data_biomedical["text"].apply(nouns))
# Visually Inspect
data_nouns.head()

,text
index,
0,thyroid surgery child institution osama ibrahi...
1,strategy year query disjoint citation query qp...
2,arterybypass thrombosis brin brinogen mutation...
3,plasmacytoma sp skull entity proliferation pla...
4,study serum matrix metalloproteinase patient t...


## Part B: The Document-Term Matrix (DTM)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# data_nouns and data_biomedical 

# Create a TF-IDF Vectorizer with specified parameters
tv_noun = TfidfVectorizer(stop_words='english', ngram_range=(1, 1), max_df=0.8, min_df=0.01)

# Fit and Transform speech noun text to a TF-IDF Doc-Term Matrix
data_tv_noun = tv_noun.fit_transform(data_nouns.text)

# Create a data-frame of Doc-Term Matrix with nouns as column names
data_dtm_noun = pd.DataFrame(data_tv_noun.toarray(), columns=tv_noun.get_feature_names_out())

# Set the index of data_dtm_noun
data_dtm_noun.index = data_biomedical.index

# Visually inspect Document Term Matrix
data_dtm_noun.head(10)


,aa,aac,aag,aat,ab,abbott,abbreviation,abc,abcam,abdomen,...,zi,zinc,zj,zl,zone,zou,zscores,zt,zy,zz
index,,,,,,,,,,,,,,,,,,,,,
0,0.013158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020768,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.092709,0.0,0.0,0.0,0.000000,0.0
5,0.014266,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.034060,0.0,0.0,0.0,0.000000,0.0
6,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
7,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.026109,0.0
8,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0


#### The rows: Each row in the DTM corresponds to a document or a piece of text of the dataset.
#### The columns: Each column corresponds to a unique term (word) found in the entire corpus (collection of documents). These terms are essentially the features used for analysis. The columns are labeled with the terms they represent, such as 'aa', 'aac', 'aag', 'aat', 'ab', 'abbott', 'abbreviation', and so on
#### The values: The values in each cell of the matrix represent the TF-IDF score of the corresponding term in the corresponding document. TF-IDF is a numerical statistic that reflects the importance of a term within a document relative to its importance in the entire corpus. It takes into account both term frequency (how often the term appears in the document) and inverse document frequency (how unique or rare the term is across the entire corpus). Higher TF-IDF scores indicate that a term is more important within a specific document.
#### furthermore, the text data, most of the terms are absent or occur infrequently in each document. This leads to a sparse matrix where most values are zero (as indicated by the many 0.0 values in your example). This sparsity is common in text data and is one reason why techniques like TF-IDF are used to represent text data efficiently.
#### Stop words have been specified stop_words='english' when creating the TF-IDF vectorizer. This means that common English stop words like "the," "and," "is," etc., have been removed from the text before vectorization. This helps in focusing on more meaningful terms.
#### Thus, each row corresponds to a document, each column to a term, and the values represent the importance of each term within each document, with common and unimportant terms filtered out.

## Part C: Run the NMF

In [16]:
def display_topics(model, feature_names, num_top_words, topic_names=None):
    '''Given an NMF model, feature_names, and number of top words, print 
       topic number and its top feature names, up to specified number of top words.'''
    # iterate through topics in topic-term matrix, 'H' aka
    # model.components_
    for ix, topic in enumerate(model.components_):
        #print topic, topic number, and top words
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i] \
             for i in topic.argsort()[:-num_top_words - 1:-1]]))

In [17]:
nmf_model = NMF(5)
# Learn an NMF model for given Document Term Matrix 'V' 
# Extract the document-topic matrix 'W'
doc_topic = nmf_model.fit_transform(data_dtm_noun)
# Extract top words from the topic-term matrix 'H' 
display_topics(nmf_model, tv_noun.get_feature_names_out(), 10)


Topic  0
expression, tumor, mir, tissue, breast, level, proliferation, migration, fig, apoptosis

Topic  1
disease, risk, group, case, surgery, lung, year, treatment, health, age

Topic  2
gene, expression, network, pathway, degs, sample, set, model, method, methylation

Topic  3
mutation, tumor, egfr, lung, alk, pfs, response, gefitinib, trial, dna

Topic  4
mouse, protein, tumor, virus, infection, response, activity, day, cytokine, activation


### From the given results seen above: 
#### Topic 0: This topic seems to be related to molecular biology or cellular biology. The top words include "expression," "tumor," "mir" (possibly microRNA), "breast," "proliferation," "migration," "apoptosis," and "fig." These terms are often associated with the study of genes, cancer, and cellular processes.
#### Topic 1: This topic appears to be related to medical or health-related discussions. The top words include "disease," "risk," "group," "case," "surgery," "lung," "year," "treatment," and "age." It may represent discussions about medical conditions, treatment options, and risk factors.
#### Topic 2: This topic seems to be related to bioinformatics or computational biology. The top words include "gene," "expression," "network," "pathway," "degs" (possibly differentially expressed genes), "sample," "set," "model," and "methylation." These terms are often used in the context of analyzing gene expression data and biological networks.
#### Topic 3: This topic appears to be related to genetics and medical research. The top words include "mutation," "tumor," "egfr" (epidermal growth factor receptor), "lung," "alk" (anaplastic lymphoma kinase), "pfs" (possibly progression-free survival), "response," "gefitinib" (a cancer drug), and "dna." This topic might be discussing genetic mutations, clinical trials, and treatment responses in the context of cancer research.
#### Topic 4: This topic seems to be related to immunology or virology. The top words include "mouse," "protein," "tumor," "virus," "infection," "response," "activity," "day," "cytokine," and "activation." It may represent discussions about immune responses to viral infections or the role of proteins and cytokines in immune reactions.
#### Thus, each document in the corpus is likely a mixture of these topics to varying degrees. For example, a medical research paper might have a higher proportion of words from Topic 3 (genetics) and a lower proportion from Topic 2 (bioinformatics). therefore, topic modeling can be a useful tool for summarizing and understanding the main themes or topics present in a large collection of text documents. It can help in organizing and categorizing documents, making it easier to navigate and analyze a large text corpus.

### The code used above is taken from: https://github.com/fenna/BFVM23DATASCNC5/blob/main/Tutorials/tutorial_clustering_words.ipynb